<a href="https://www.kaggle.com/code/arshaqshagihan/model-function?scriptVersionId=235290067" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fork-of-fake-news-detection-using-cnn-78099a/__results__.html
/kaggle/input/fork-of-fake-news-detection-using-cnn-78099a/__notebook__.ipynb
/kaggle/input/fork-of-fake-news-detection-using-cnn-78099a/__output__.json
/kaggle/input/fork-of-fake-news-detection-using-cnn-78099a/model_weights.npy
/kaggle/input/fork-of-fake-news-detection-using-cnn-78099a/vectorizer.pkl
/kaggle/input/fork-of-fake-news-detection-using-cnn-78099a/custom.css
/kaggle/input/fork-of-fake-news-detection-using-cnn-78099a/__results___files/__results___22_1.png
/kaggle/input/fork-of-fake-news-detection-using-cnn-78099a/__results___files/__results___44_1.png


In [2]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(100, activation='relu', input_dim=1000),
    Dense(50, activation='relu'),
    Dense(25, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid')
])

#Proper weight loading for .npy files
weights_path = '/kaggle/input/fork-of-fake-news-detection-using-cnn-78099a/model_weights.npy'

# Load all weights as a flat list
all_weights = np.load(weights_path, allow_pickle=True)

# Verify total weights match architecture (5 dense layers × 2 = 10 weight arrays)
if len(all_weights) != 10:
    raise ValueError(f"Expected 10 weight arrays, got {len(all_weights)}")

# Set weights in pairs (kernel + bias) for each layer
weight_index = 0
for layer in model.layers:
    # Each dense layer needs 2 weight arrays
    layer_weights = all_weights[weight_index:weight_index+2]
    layer.set_weights(layer_weights)
    weight_index += 2

print("Weights loaded successfully!")
print(f"First layer weights shape: {model.layers[0].get_weights()[0].shape}")
print(f"First layer bias shape: {model.layers[0].get_weights()[1].shape}")


Weights loaded successfully!
First layer weights shape: (1000, 100)
First layer bias shape: (100,)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [3]:
import pickle

try:
    # Open the file in binary read mode ('rb')
    with open('/kaggle/input/fork-of-fake-news-detection-using-cnn-78099a/vectorizer.pkl', 'rb') as f:
        vectorizer = pickle.load(f)  # Pass the file object 'f' to pickle.load()
    print("Vectorizer loaded successfully!")
    # Now you can use 'vectorizer'
except FileNotFoundError:
    print("Error: Vectorizer file not found.")
except TypeError as e:
    print(f"Type Error: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

Vectorizer loaded successfully!


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.2.post1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
import numpy as np
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import nltk
from nltk.corpus import stopwords
import tensorflow as tf

# Download stopwords if not already available
nltk.download('stopwords', quiet=True)
stop_words = set(stopwords.words('english'))

def preprocess(text):
    """Basic text cleaning: lowercases, removes stopwords, and non-alphabetical words."""
    return ' '.join([word.lower() for word in text.split() 
                     if word.lower() not in stop_words and word.isalpha()])


# Build the model architecture as you provided
model = Sequential([
    Dense(100, activation='relu', input_dim=1000),
    Dense(50, activation='relu'),
    Dense(25, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Load weights manually from file (ensure the file path is correct in your environment)
weights = np.load('/kaggle/input/fork-of-fake-news-detection-using-cnn-78099a/model_weights.npy', allow_pickle=True)
for layer, (w, b) in zip(model.layers, weights.reshape(-1, 2)):
    layer.set_weights([w, b])

def integrated_gradients(model, x, baseline=None, steps=50):
    """
    Compute Integrated Gradients for a given input x of shape (1, features).
    
    Args:
        model: The trained model.
        x: Input numpy array of shape (1, features) for which to compute attributions.
        baseline: Baseline input (if None, defaults to a zero-vector of same shape as x).
        steps: Number of interpolation steps between baseline and input.
    
    Returns:
        Integrated gradients as a numpy array of shape (1, features).
    """
    if baseline is None:
        baseline = np.zeros(x.shape, dtype=np.float32)
    
    # Generate scaled inputs between the baseline and the actual input.
    scaled_inputs = [
        baseline + (float(i) / steps) * (x - baseline)
        for i in range(steps + 1)
    ]
    # Convert to TensorFlow tensor
    scaled_inputs = tf.convert_to_tensor(np.array(scaled_inputs), dtype=tf.float32)
    
    # At this point, scaled_inputs has shape (steps+1, 1, 1000).
    # Remove the extra dimension to match the input shape expected by the model.
    scaled_inputs = tf.squeeze(scaled_inputs, axis=1)  # Now shape: (steps+1, 1000)
    
    with tf.GradientTape() as tape:
        tape.watch(scaled_inputs)
        predictions = model(scaled_inputs)
    grads = tape.gradient(predictions, scaled_inputs)
    
    # Average the gradients across the steps.
    avg_grads = tf.reduce_mean(grads, axis=0).numpy()  # Shape: (1000,)
    
    # Compute the integrated gradients
    # Note: We need to remove the extra batch dimension from x and baseline.
    integrated_grad = (x[0] - baseline[0]) * avg_grads  # Shape: (1000,)
    
    # Add back a batch dimension (if needed)
    return integrated_grad[np.newaxis, :]

def analyze_article(article, top_n=5):
    """
    Analyzes an article by preprocessing, vectorizing, predicting,
    and computing Integrated Gradients to identify key words.
    
    Args:
        article: Text string representing the article.
        top_n: Number of top contributing keywords to return.
    
    Returns:
        A dictionary with prediction, confidence, and a list of top keywords with their IG scores.
    """
    # Preprocess the input text
    processed = preprocess(article)
    
    # Transform the text using the vectorizer.
    bow_vector = vectorizer.transform([processed]).toarray()
    
    # Pad the vector to have exactly 1000 features (CountVectorizer might output fewer features)
    padded_bow = np.zeros((1, 1000))
    padded_bow[0, :bow_vector.shape[1]] = bow_vector

    # Get the model's prediction and compute the confidence.
    confidence = model.predict(padded_bow, verbose=0)[0][0]
    prediction = 'Fake' if confidence > 0.5 else 'Real'
    adjusted_confidence = float(confidence if prediction == 'Fake' else 1 - confidence)
    
    # Compute Integrated Gradients for feature importance.
    ig_values = integrated_gradients(model, padded_bow.astype(np.float32))
    ig_values = ig_values[0]  # Remove the batch dimension
    
    # Map each word that actually appears in the article to its IG attribution.
    feature_names = vectorizer.get_feature_names_out()
    non_zero_indices = np.nonzero(bow_vector[0])[0]
    keywords_with_ig = [(feature_names[i], ig_values[i]) for i in non_zero_indices]
    
    # Sort the keywords by the absolute attribution value and return the top_n.
    sorted_keywords = sorted(keywords_with_ig, key=lambda x: abs(x[1]), reverse=True)[:top_n]
    
    return {
        'prediction': prediction,
        'confidence': adjusted_confidence,
        'keywords': sorted_keywords
    }


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_5 (Dense)                      │ (None, 100)                 │         100,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 50)                  │           5,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 25)                  │           1,275 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 10)                  │             260 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │              11 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 106,696 (416.78 KB)

 Trainable params: 106,696 (416.78 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
article = """"Global Leaders Convene Amidst Growing Calls for Unified AI Regulation

SAN FRANCISCO, CA – In a series of high-level meetings held across continents, global leaders and technology executives are accelerating efforts to establish a unified framework for artificial intelligence (AI) regulation. The urgency stems from the rapid advancement of AI technologies, which many believe necessitate immediate international cooperation to mitigate potential risks.

Discussions have centered on the need for ethical guidelines that ensure AI systems are developed and deployed responsibly. Concerns have been raised regarding the potential for AI to amplify societal biases, spread misinformation, and disrupt labor markets. Participants emphasized the importance of transparency in AI algorithms, advocating for mechanisms that allow for public scrutiny and accountability.

'We are at a critical juncture,' stated a European Union delegate during a closed-door session. 'The decisions we make today will shape the trajectory of AI development for generations. It is imperative that we establish common standards that prioritize human safety and well-being.'

The United States, while expressing support for international collaboration, has also stressed the importance of fostering innovation. 'We must strike a balance between regulation and innovation,' remarked a US official. 'Overly restrictive measures could stifle the very advancements that hold immense potential for societal benefit.'

One of the most contentious issues remains the regulation of open-source AI models. Some argue that open-source AI should be subject to the same regulatory oversight as proprietary systems, while others contend that such measures would hinder the open exchange of knowledge and innovation.

The formation of an international AI regulatory body has also been proposed, tasked with developing and enforcing global standards. However, the exact structure and mandate of such a body remain subjects of debate.

As the discussions continue, the world watches closely, aware that the outcome will have profound implications for the future of AI and its role in society.
"""


In [7]:
result = analyze_article(article)
print(f"Prediction: {result['prediction']} ({result['confidence']:.1%} confidence)")
print("Key factors:")
for word, score in result['keywords']:
    print(f" - {word}: {score:.4f}")


Prediction: Real (51.9% confidence)
Key factors:
 - global: 0.2641
 - role: 0.1133
 - make: 0.0946
 - establish: -0.0929
 - us: -0.0775


In [8]:
article ='''Trump Declares Himself "King of Florida" in Surprise Beachside Coronation

April 21, 2025 — Mar-a-Lago, Florida

In an unexpected move early Monday morning, former President Donald J. Trump emerged from his Mar-a-Lago estate wearing a velvet robe, a golden crown made from repurposed golf trophies, and declared himself the “Rightful and Eternal King of Florida.”

“I’ve always said I was a winner,” Trump announced from a sandcastle podium adorned with miniature American flags. “And now, I’ve won Florida. Totally legally. Ask anyone. Ron DeSantis tried, but I’m just more royal.”

Supporters gathered along Palm Beach in what appeared to be a spontaneous cosplay rally, many dressed as court jesters, knights, and “MAGA monarchs.” One attendee held a sign reading "Make Feudalism Great Again."

Trump's royal decree included plans for:

    A new "Trump Tower Capitol" built entirely from golden seashells and optimism.

    Free Diet Coke fountains every ten feet.

    The renaming of Disney World to “Trumplandia.”

Political analysts are unsure how seriously to take the announcement. Governor DeSantis responded by issuing a statement reading simply: “...what?”

When asked about legal implications, Trump waved dismissively. “The Constitution doesn’t say I can’t be king. I checked.”

The Kingdom of Florida now awaits recognition from international bodies, or at least an endorsement from Kanye West..'''

In [9]:
result = analyze_article(article)
print(f"Prediction: {result['prediction']} ({result['confidence']:.1%} confidence)")
print("Key factors:")
for word, score in result['keywords']:
    print(f" - {word}: {score:.4f}")


Prediction: Fake (57.0% confidence)
Key factors:
 - said: 0.1036
 - former: -0.0678
 - donald: -0.0629
 - made: 0.0598
 - american: 0.0575
